# Goal: 
Construct, analyze, and publish a dataset of monthly traffic on English Wikipedia from January 1, 2008 through August 30, 2021

# Data Acquisition

There are 2 sources of Wikipedia Traffic data
    - Pagecounts
    - Pageviews
The main difference between pagecounts and the pageview data is that pageview allows us to filter out automated (bot) traffic whereas pagecounts does not.

We will be collecting traffic data from 2008 - 2021 & outputting to 5 different files
- pagecounts_desktop-site_200801-201607.json
- pagecounts_mobile-site_200801-201607.json
- pageviews_desktop_201507-202108.json
- pageviews_mobile-app_201507-202108.json
- pageviews_mobile-web_201507-202108.json

In [1]:
import json
import requests
import pandas as pd
import altair as alt
from altair_saver import save
from selenium import webdriver

In [2]:
endpoint_legacy = 'https://wikimedia.org/api/rest_v1/metrics/legacy/pagecounts/aggregate/{project}/{access-site}/{granularity}/{start}/{end}'
endpoint_pageviews = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/aggregate/{project}/{access}/{agent}/{granularity}/{start}/{end}'

In [3]:

#API call params

# see: https://wikimedia.org/api/rest_v1/#!/Legacy_data/get_metrics_legacy_pagecounts_aggregate_project_access_site_granularity_start_end
pagecounts_desktop_site = {"project" : "en.wikipedia.org",
                 "access-site" : "desktop-site",
                 "granularity" : "monthly",
                 "start" : "2008010100",
                # for end use 1st day of month following final month of data
                 "end" : "2016080100"
                    }

pagecounts_mobile_site = {"project" : "en.wikipedia.org",
                 "access-site" : "mobile-site",
                 "granularity" : "monthly",
                 "start" : "2008010100",
                # for end use 1st day of month following final month of data
                 "end" : "2016080100"
                    }

# parameters for getting aggregated current standard pageview data
# see: https://wikimedia.org/api/rest_v1/#!/Pageviews_data/get_metrics_pageviews_aggregate_project_access_agent_granularity_start_end
pageviews_desktop = {"project" : "en.wikipedia.org",
                    "access" : "desktop",
                    "agent" : "user",
                    "granularity" : "monthly",
                    "start" : "2015070100",
                    # for end use 1st day of month following final month of data
                     "end" : "2021090100"
                        }
pageviews_mobile_app = {"project" : "en.wikipedia.org",
                    "access" : "mobile-app",
                    "agent" : "user",
                    "granularity" : "monthly",
                    "start" : "2015070100",
                    # for end use 1st day of month following final month of data
                     "end" : "2021090100"
                        }

pageviews_mobile_web = {"project" : "en.wikipedia.org",
                    "access" : "mobile-web",
                    "agent" : "user",
                    "granularity" : "monthly",
                    "start" : "2015070100",
                    # for end use 1st day of month following final month of data
                     "end" : "2021090100"
                        }
# Customize these with your own information
headers = {
    'User-Agent': 'https://github.com/marcm97',
    'From': 'marcm5@uw.edu'
}

In [4]:
def api_call(endpoint,parameters):
    call = requests.get(endpoint.format(**parameters), headers=headers)
    response = call.json()
    
    return response

In [5]:
#pagecounts API calls
pagecounts_desktop_site_json = api_call(endpoint_legacy, pagecounts_desktop_site)
pagecounts_mobile_site_json = api_call(endpoint_legacy, pagecounts_mobile_site)

#pageview API calls
pageviews_desktop_json = api_call(endpoint_pageviews, pageviews_desktop)
pageviews_mobile_app_json = api_call(endpoint_pageviews, pageviews_mobile_app)
pageviews_mobile_web_json = api_call(endpoint_pageviews, pageviews_mobile_web)

In [6]:
# writing json data to data_raw/
with open('data_raw/pagecounts_desktop-site_200801-201608.json', 'w', encoding='utf-8') as f:
    json.dump(pagecounts_desktop_site_json, f, ensure_ascii=False, indent=4)
    
with open('data_raw/pagecounts_mobile-site_200801-201608.json', 'w', encoding='utf-8') as f:
    json.dump(pagecounts_mobile_site_json, f, ensure_ascii=False, indent=4)
    
with open('data_raw/pageviews_desktop_201507-202108.json', 'w', encoding='utf-8') as f:
    json.dump(pageviews_desktop_json, f, ensure_ascii=False, indent=4)

with open('data_raw/pageviews_mobile-app_201507-202108.json', 'w', encoding='utf-8') as f:
    json.dump(pageviews_mobile_app_json, f, ensure_ascii=False, indent=4)
    
with open('data_raw/pageviews_mobile-web_201507-202108.json', 'w', encoding='utf-8') as f:
    json.dump(pageviews_mobile_web_json, f, ensure_ascii=False, indent=4)

# Data Processing

In [7]:
def create_df(path):
    with open(path) as f:
        json_data = json.load(f)
    data = pd.json_normalize(json_data["items"])
    data["Year"] = data["timestamp"].str[:4]#first 4 digits are the year
    data["Month"] = data["timestamp"].str[4:6]#digits 5,6 are months
    if "views" in data.columns:
        data["count"] = data["views"]
    data["count"] = pd.to_numeric(data["count"])
    return data[["Year","Month","count"]]
    

In [8]:
pagecounts_desktop = create_df("data_raw/pagecounts_desktop-site_200801-201608.json")
pagecounts_mobile = create_df("data_raw/pagecounts_mobile-site_200801-201608.json")
pageviews_desktop = create_df("data_raw/pageviews_desktop_201507-202108.json")
pageviews_mobile_app = create_df("data_raw/pageviews_mobile-app_201507-202108.json")
pageviews_mobile_web = create_df("data_raw/pageviews_mobile-web_201507-202108.json")

- For data collected from the Pageviews API, combine the monthly values for mobile-app and mobile-web to create a total mobile traffic count for each month.


In [9]:
pageviews_mobile = pageviews_mobile_app.merge(pageviews_mobile_web,
                          how = "outer",
                           on =["Year","Month"],
                           suffixes =["_mobile_app","_mobile_web"]
                          )
pageviews_mobile["count"] = pageviews_mobile["count_mobile_app"]+pageviews_mobile["count_mobile_web"]
pageviews_mobile = pageviews_mobile[["Year","Month","count"]]

In [10]:
#creating unified joined csv
pagecounts = pd.merge(pagecounts_desktop,
         pagecounts_mobile,
         how = "outer",
         on =["Year","Month"],
         suffixes =["_pagecounts_desktop","_pagecounts_mobile_web"]
        )

pageviews = pd.merge(pageviews_desktop,
         pageviews_mobile,
         how = "outer",
         on =["Year","Month"],
         suffixes =["_pageview_desktop","_pageview_mobile_web"]
        )

data = pd.merge(pagecounts,
                pageviews,
                how = "outer",
                on =["Year","Month"]
               )

In [11]:
#replacing NA with 0's
data = data.fillna(0)

In [12]:
data["pagecount_all_views"] = data["count_pagecounts_desktop"] + data["count_pagecounts_mobile_web"]
data["pageview_all_views"] = data["count_pageview_desktop"] + data["count_pageview_mobile_web"]

In [13]:
data = data.rename(columns = {
    "count_pagecounts_desktop": "pagecount_desktop_views",
    "count_pagecounts_mobile_web": "pagecount_mobile_views",
    "count_pageview_desktop": "pageview_desktop_views",
    "count_pageview_mobile_web":"pageview_mobile_views"
})
data = data[["Year",
      "Month",
      "pagecount_all_views",
      "pagecount_desktop_views",
      "pagecount_mobile_views",
      "pageview_all_views",
      "pageview_desktop_views",
      "pageview_mobile_views"
     ]]

In [14]:
# write out to csv
data.to_csv("data_clean/en-wikipedia_traffic_200712-202108.csv")

# Data Analysis

Your visualization will track three traffic metrics: mobile traffic, desktop traffic, and all traffic (mobile + desktop).
In order to complete the analysis correctly and receive full credit, your graph will need to be the right scale to view the data; all units, axes, and values should be clearly labeled. Your graph should possess a legend and a title. You must generate a .png or .jpeg formatted image of your final graph.


In [15]:
data["date"] = pd.to_datetime(data["Year"].astype('str') + data["Month"].astype('str'),format='%Y%m')

In [16]:
melted_data = pd.melt(data,
        id_vars = ["Year","Month","date"],
        var_name = "View Type",
        value_name = "View Count"
       )

In [17]:
melted_data

,Year,Month,date,View Type,View Count
0,2008,01,2008-01-01,pagecount_all_views,4.930903e+09
1,2008,02,2008-02-01,pagecount_all_views,4.818394e+09
2,2008,03,2008-03-01,pagecount_all_views,4.955406e+09
3,2008,04,2008-04-01,pagecount_all_views,5.159162e+09
4,2008,05,2008-05-01,pagecount_all_views,5.584691e+09
...,...,...,...,...,...
979,2021,04,2021-04-01,pageview_mobile_views,4.759095e+09
980,2021,05,2021-05-01,pageview_mobile_views,4.976580e+09
981,2021,06,2021-06-01,pageview_mobile_views,4.584510e+09
982,2021,07,2021-07-01,pageview_mobile_views,4.778909e+09


In [25]:
chart = alt.Chart(melted_data[melted_data["View Count"]!=0],title = "Wikipedia Traffic (Jan 2008 - Oct 2021)").mark_line(opacity=1).encode(
    x = alt.X("yearmonth(date)",title = "Date"),
    y = "View Count:Q",
    color = "View Type",   
)

In [26]:
chart

alt.Chart(...)

In [22]:
save(chart,"results/wikipedia_traffic.png")